In [7]:
import paramiko

# 서버 접속 정보
ip = "172.16.16.24"
user = "root"
pw = "asd123!@"

# 고정된 경로
HTTPD_CONF = "/etc/apache2/sites-available/000-default.conf"
VHOST_DIR = "/etc/apache2/sites-available"
APACHE2_CONF = "/etc/apache2/apache2.conf"

# 동적으로 받을 값
WEB_DOCUMENT_ROOT = ""
VHOST_CONF = ""
SERVER_NAME = ""
ERROR_LOG = ""
CUSTOM_LOG = ""


def cc(cmd):
    """SSH로 명령어를 실행하고 결과 출력"""
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()


def set_ufw():
    """방화벽 상태 확인 및 중지"""
    print("🔥 방화벽 설정 중...")
    cmd = """
systemctl stop ufw
systemctl disable ufw
echo 'UFW 방화벽 중지 완료'
"""
    cc(cmd)


def install_package():
    """Apache 설치"""
    print("📦 Apache 설치 중...")
    cmd = """
apt update -y
apt install -y apache2
"""
    cc(cmd)


def start_apache():
    """Apache 실행"""
    print("🚀 Apache 실행 중...")
    cmd = """
systemctl enable apache2
systemctl restart apache2
systemctl status apache2 --no-pager
"""
    cc(cmd)


def set_user_dir_and_vhost_conf():
    """사용할 웹 디렉토리 입력"""
    global WEB_DOCUMENT_ROOT
    WEB_DOCUMENT_ROOT = input("사용할 웹 디렉토리를 입력하세요 (예: /home/team1 또는 /var/www/young): ").strip()
    cc(f"mkdir -p {WEB_DOCUMENT_ROOT} && chmod 755 {WEB_DOCUMENT_ROOT}")
    print(f"✅ 웹 문서 루트: {WEB_DOCUMENT_ROOT}")

def set_apache2_conf() :
    cmd = f"""
    sed -i 's|<Directory /var/www/>|<Directory {WEB_DOCUMENT_ROOT}>|' {APACHE2_CONF}
    """
    cc(cmd)

def set_vhostconf():
    """가상 호스트 설정 파일 생성"""
    global VHOST_CONF, SERVER_NAME, ERROR_LOG, CUSTOM_LOG

    vhost_name = input("VHost 설정 파일 이름을 입력하세요 (예: team1.conf): ")
    VHOST_CONF = f"{VHOST_DIR}/{vhost_name}"

    SERVER_NAME = input("ServerName을 입력하세요 (예: test.local): ")
    ERROR_LOG = input("ErrorLog 경로를 입력하세요 (예: /var/log/apache2/team1_error.log): ")
    CUSTOM_LOG = input("CustomLog 경로를 입력하세요 (예: /var/log/apache2/team1_access.log): ")

    print(f"🛠️ {VHOST_CONF} 생성 중...")

    cmd = f"""
cat <<EOF > {VHOST_CONF}
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName {SERVER_NAME}
    DocumentRoot {WEB_DOCUMENT_ROOT}

    <Directory {WEB_DOCUMENT_ROOT}>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

    ErrorLog {ERROR_LOG}
    CustomLog {CUSTOM_LOG} combined
</VirtualHost>
EOF

a2ensite {vhost_name}
a2dissite 000-default.conf
systemctl reload apache2
mkdir -p {ERROR_LOG}; mkdir -p {CUSTOM_LOG};
echo '✅ VirtualHost 설정 완료: {VHOST_CONF}'
"""
    cc(cmd)


def mk_index():
    """기본 index.html 생성"""
    print("📄 index.html 생성 중...")
    cmd = f"echo '<h1>This is index.html for Test by mk_index func</h1>' > {WEB_DOCUMENT_ROOT}/index.html"
    print(f"{WEB_DOCUMENT_ROOT} 경로에 파일이 생성되었습니다")
    cc(cmd)


def restart_apa():
    """Apache 재시작"""
    print("🔁 Apache 재시작 중...")
    cc("systemctl restart apache2 && echo 'Apache 재시작 완료'")

def install_ftp():
    cc("apt install -y vsftpd")

def start_ftp():
    cc("systemctl enable vsftpd; systemctl start vsftpd")

def go():
    """전체 자동화 실행"""
#    set_ufw()
#    install_package()
#    start_apache()
    set_user_dir_and_vhost_conf()
    set_apache2_conf()
#    set_vhostconf()
#    mk_index()
#    restart_apa()
    print("🎉 모든 작업이 완료되었습니다!")

    install_ftp()
    start_ftp()
if __name__ == "__main__":
    go()


사용할 웹 디렉토리를 입력하세요 (예: /home/team1 또는 /var/www/young):  /home/test



✅ 웹 문서 루트: /home/test

🎉 모든 작업이 완료되었습니다!
